In [ ]:
import numpy as np
from collections import defaultdict
import heapq

class Grafo:
    def __init__(self, n_vertices):
        self.ordem = n_vertices
        self.tamanho = 0
        self.listas_adjacencias = defaultdict(list)

    def adicionaVertice(self, u):
        if not self.temVertice(u):
            self.listas_adjacencias[u] = []
        else:
            print(f"O vértice ({u}) já existe.")

    def adicionaAresta(self, u, v, peso):
        if not self.temVertice(u):
            self.adicionaVertice(u)
        if not self.temAresta(u, v):
            self.listas_adjacencias[u].append((v, peso))
            self.listas_adjacencias[v].append((u, peso))  # Para garantir que o grafo seja não direcionado
        else:
            print(f"A aresta ({u}, {v}) já existe.")

    def temVertice(self, u):
        return u in self.listas_adjacencias

    def temAresta(self, u, v):
        return any(v == aresta[0] for aresta in self.listas_adjacencias[u])

    def removeAresta(self, u, v):
        self.listas_adjacencias[u] = [aresta for aresta in self.listas_adjacencias[u] if aresta[0] != v]
        self.listas_adjacencias[v] = [aresta for aresta in self.listas_adjacencias[v] if aresta[0] != u]

    def grauEntrada(self, u):
        grau = 0
        for i, v in self.listas_adjacencias.items():
            for v, peso in v:
                if v == u:
                    grau += 1
        return grau

    def grauSaida(self, u):
        return len(self.listas_adjacencias[u]) if u in self.listas_adjacencias else 0

    def grauTotal(self, u):
        return self.grauEntrada(u) + self.grauSaida(u)

    def getPeso(self, u, v):
        if u in self.listas_adjacencias:
            for j, peso in self.listas_adjacencias[u]:
                if j == v:
                    return peso

    def imprimeMatriz(self):
        print("Lista de Adjacências: ")
        for vertice, vizinhos in self.listas_adjacencias.items():
            if vizinhos:
                arestas = " -> ".join([f"('{v}', {peso})" for v, peso in vizinhos])
                print(f"{vertice}: {arestas} ->")
            else:
                print(f"{vertice}:")

    def dfs_util(self, v, visitado, pilha):
        visitado[v] = True
        pilha.append(v)
        for vizinho, _ in self.listas_adjacencias[v]:
            if not visitado[vizinho]:
                self.dfs_util(vizinho, visitado, pilha)

    def numeroComponentes(self, direcionado=False):
        visitado = {v: False for v in self.listas_adjacencias}
        componentes = 0

        def ordemPreenchida(v, visitado, pilha):
            visitado[v] = True
            for vizinho, _ in self.listas_adjacencias[v]:
                if not visitado[vizinho]:
                    ordemPreenchida(vizinho, visitado, pilha)
            pilha.append(v)

        def transpor():
            g_transposto = Grafo(self.ordem)
            for v in self.listas_adjacencias:
                for vizinho, peso in self.listas_adjacencias[v]:
                    g_transposto.adicionaAresta(vizinho, v, peso)
            return g_transposto

        if direcionado:
            pilha = []
            for v in self.listas_adjacencias:
                if not visitado[v]:
                    ordemPreenchida(v, visitado, pilha)

            gr = transpor()
            visitado = {v: False for v in self.listas_adjacencias}

            while pilha:
                v = pilha.pop()
                if not visitado[v]:
                    gr.dfs_util(v, visitado, [])
                    componentes += 1
        else:
            for v in self.listas_adjacencias:
                if not visitado[v]:
                    self.dfs_util(v, visitado, [])
                    componentes += 1

        return componentes

    def eCiclico(self, direcionado=False):
        visitado = {v: False for v in self.listas_adjacencias}
        pilha_rec = {v: False for v in self.listas_adjacencias}

        def eCiclicoUtil(v):
            visitado[v] = True
            pilha_rec[v] = True
            for vizinho, _ in self.listas_adjacencias[v]:
                if not visitado[vizinho]:
                    if eCiclicoUtil(vizinho):
                        return True
                elif pilha_rec[vizinho]:
                    return True
            pilha_rec[v] = False
            return False

        if direcionado:
            for node in self.listas_adjacencias:
                if not visitado[node]:
                    if eCiclicoUtil(node):
                        return True
            return False
        else:
            parent = {v: None for v in self.listas_adjacencias}

            def eCiclicoNaoDirecionado(v):
                visitado[v] = True
                for vizinho, _ in self.listas_adjacencias[v]:
                    if not visitado[vizinho]:
                        parent[vizinho] = v
                        if eCiclicoNaoDirecionado(vizinho):
                            return True
                    elif parent[v] != vizinho:
                        return True
                return False

            for node in self.listas_adjacencias:
                if not visitado[node]:
                    if eCiclicoNaoDirecionado(node):
                        return True
            return False

    def prim(self):
        mst = []
        visitado = {v: False for v in self.listas_adjacencias}
        pq = []

        def adicionarArestas(v):
            visitado[v] = True
            for vizinho, peso in self.listas_adjacencias[v]:
                if not visitado[vizinho]:
                    heapq.heappush(pq, (peso, v, vizinho))

        # Começa do vértice 0 (pode ser qualquer vértice)
        adicionarArestas(0)

        while pq:
            peso, u, v = heapq.heappop(pq)
            if not visitado[v]:
                visitado[v] = True
                mst.append((u, v, peso))
                adicionarArestas(v)

        return mst


In [ ]:
G = Grafo(5)
G.adicionaVertice("Pedro")
G.adicionaVertice("Maria")
G.adicionaVertice("Luis")
G.adicionaVertice("Leiticia")
G.adicionaAresta("Pedro", "Maria", 30)
G.adicionaAresta("Vitor", "Pedro", 15)
G.adicionaAresta("Pedro", "Vitor", 4)
G.adicionaAresta("Pedro", "Luis", 9)
G.adicionaAresta("Pedro", "Leticia", 18)
G.adicionaAresta("Leticia", "Maria", 7)
G.adicionaAresta("Luis", "Pedro", 50)
G.adicionaAresta("Pedro", "Maria", 4)

A aresta (Pedro, Vitor) já existe.
A aresta (Luis, Pedro) já existe.
A aresta (Pedro, Maria) já existe.


In [ ]:
g = Grafo(5)
g.adicionaAresta(0, 1, 10)
g.adicionaAresta(0, 4, 5)
g.adicionaAresta(1, 2, 1)
g.adicionaAresta(4, 1, 3)
g.adicionaAresta(4, 2, 9)
g.adicionaAresta(4, 3, 2)
g.adicionaAresta(2, 3, 4)
g.adicionaAresta(3, 2, 6)

mst = g.prim()
print("Árvore Geradora Mínima (MST):")
for u, v, peso in mst:
    print(f"Aresta ({u}, {v}) com peso {peso}")


A aresta (3, 2) já existe.
Árvore Geradora Mínima (MST):
Aresta (0, 4) com peso 5
Aresta (4, 3) com peso 2
Aresta (4, 1) com peso 3
Aresta (1, 2) com peso 1
